In [23]:
import os
import numpy as np
import nibabel as nib
from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.align.imaffine import MutualInformationMetric, AffineRegistration
from dipy.align.transforms import RigidTransform3D
from dipy.viz import regtools


import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nibabel as nib
from nibabel.processing import resample_from_to
from nilearn import plotting

from dipy.reconst.dti import TensorModel, fractional_anisotropy, mean_diffusivity, radial_diffusivity, axial_diffusivity
from dipy.io.image import load_nifti_data
from dipy.core.gradients import gradient_table

from dipy.align.imaffine import AffineMap
from dipy.align.metrics import CCMetric
from dipy.align.imwarp import SymmetricDiffeomorphicRegistration

from dipy.align.imaffine import AffineRegistration
from dipy.align.transforms import RigidTransform3D
from dipy.align.metrics import SSDMetric
from dipy.align.imaffine import AffineRegistration, MutualInformationMetric

import numpy as np
import nibabel as nib
from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.align import motion_correction

from dipy.core.gradients import rotate_bvecs


In [7]:
main_dir = "/media/ist/Drive2/MANSOOR/Neuroimaging-Project/DTI-Analysis"
data_dir = f"{main_dir}/Data/test/test-sub-01-pre"
save_prep_dir = f"{data_dir}/prep-dti"
results_dir = f"{main_dir}/results"
static_img_file = f"{data_dir}/target.nii.gz"

filename = "sub_01_pre"

In [41]:

# load the DTI data
img_file = f'{data_dir}/{filename}.nii.gz'
bvals_file = f'{data_dir}/{filename}.bval'
bvecs_file = f'{data_dir}/{filename}.bvec'

data, affine = load_nifti(img_file)
bvals, bvecs = read_bvals_bvecs(bvals_file, bvecs_file)
gtab = gradient_table(bvals, bvecs)


In [39]:

# Step 1: Skull stripping using median otsu method
# Assuming 'data' is your 4D DTI dataset and you want to use the first volume for skull stripping
# Here '0' is the index of the b0 volume which is typically used for skull stripping
data, mask = median_otsu(data, vol_idx=[0], median_radius=2, numpass=1)

# Save skull-stripped data and mask
ss_data_img = nib.Nifti1Image(data, affine)
ss_mask_img = nib.Nifti1Image(mask.astype(np.uint8), affine)
save_nifti(f'{save_prep_dir}/{filename}-skull_stripped.nii.gz', data, affine)
save_nifti(f'{save_prep_dir}/{filename}-brain_mask.nii.gz', mask.astype(np.uint8), affine)



In [47]:
data_small = data[..., :3]
bvals_small = bvals[:3]
bvecs_small = bvecs[:3]
gtab = gradient_table(bvals_small, bvecs_small)

data_corrected, reg_affines = motion_correction(data_small, gtab, affine)


Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]


In [49]:
data_corrected.shape

(110, 110, 64, 3)

In [ ]:
# Apply motion correction
corrected_data, transformed_affines = motion_correction(data, gtab, affine)

# Save the motion-corrected data
save_nifti(f'{save_prep_dir}/{filename}-motion_corrected.nii.gz', corrected_data.get_fdata(), transformed_affines.affine)

# Update the bvecs to account for rotation
corrected_bvecs = rotate_bvecs(bvecs, transformed_affines)

# Save the corrected bvecs
np.savetxt(f'{save_prep_dir}/{filename}-corrected_bvecs.bvec', corrected_bvecs)

print("Motion correction complete. Corrected data and gradient vectors saved.")


In [25]:
# Step 2: Simple motion correction - align each volume to the b0 image

# Assuming 'data' is your 4D DTI dataset and 'affine' is the affine matrix of the image
# Static image (reference image)
# Initialize metric and registration object
metric = MutualInformationMetric(nbins=32)
reg = AffineRegistration(metric=metric, level_iters=[10000, 1000, 100], sigmas=[3.0, 1.0, 0.1], factors=[4, 2, 1])

# Select a transform type
transform = RigidTransform3D()

# Prepare to store registered volumes
registered_data = np.zeros_like(data)

# Static image for registration (average of b0 images)
static = data[..., gtab.b0s_mask].mean(axis=3)

# Perform registration for each volume
for i in range(data.shape[-1]):
    moving = data[..., i]
    affine_map = reg.optimize(static, moving, transform, params0=None, static_grid2world=affine, moving_grid2world=affine)
    registered_data[..., i] = affine_map.transform(moving)

# Save the motion corrected data
save_nifti(f'{save_prep_dir}/{filename}-motion_corrected.nii.gz', registered_data, affine)



Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter: 10000]
Optimizing level 1 [max iter: 1000]
Optimizing level 0 [max iter: 100]
Optimizing level 2 [max iter

KeyboardInterrupt: 

In [ ]:
# Step 3: Affine registration to a standard space (example)
# Note: You need a standard space image loaded as `static`, and `static_affine`

b0 = data[..., gtab.b0s_mask]
static, static_affine = load_nifti(static_img_file)

# Registration
affreg = AffineRegistration(metric=MutualInformationMetric(32, 'JointHistogram'))
transform = RigidTransform3D()
params0 = None
affine = affreg.optimize(static, b0, transform, params0, static_affine, affine)

# Apply transformation
transformed = affine.transform(b0)

# Save the registered image
save_nifti(f'{save_prep_dir}/{filename}-registered_to_standard.nii.gz', transformed, static_affine)

print("DTI preprocessing complete.")
